In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import cv2
import math
from scipy.cluster.hierarchy import fclusterdata

df_left_image = pd.read_csv('journal_dataset/stationary_camera_2020-06-24-15-49-10-001/stereo-left-image.csv', nrows = 10)
df_left_image.head()
raw_string = df_left_image['data'][0]


In [3]:
np.random.rand(6)

array([0.90750437, 0.47539168, 0.74267096, 0.7503687 , 0.32864623,
       0.71961623])

In [3]:
df_left_image.head()

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,height,width,encoding,is_bigendian,step,data
0,1.593039e+09,112559,1593038951,47999498,NaN,1080,1920,rgb8,0,5760,"b' \'\n""*\x0c$\'\x14\x1f#\x0f\x1d!\x12 %\x16\x..."
1,1.593039e+09,112560,1593038951,81345327,NaN,1080,1920,rgb8,0,5760,b'\x19)\n\x15%\x07 (\x0f\x1d&\x0c\x16$\x18\x15...
2,1.593039e+09,112561,1593038951,114722989,NaN,1080,1920,rgb8,0,5760,b'\x1a)\x04\x17&\x01\x17&\x13\x16%\x12\x1f#\x1...
3,1.593039e+09,112562,1593038951,148102996,NaN,1080,1920,rgb8,0,5760,"b'\x18%\x03\x19&\x04!$\x03\x1f""\x01\x1e#\x11\x..."
4,1.593039e+09,112563,1593038951,181353277,NaN,1080,1920,rgb8,0,5760,b'\x1b\'\x07\x18$\x05\x1b#\x17\x1d&\x1a##\x18%...


In [2]:
raw_string[0:25]

'b\' \\\'\\n"*\\x0c$\\\'\\x14\\x1f#'

In [ ]:

byte_string = raw_string[2:-1].encode('latin1')
escaped_string = byte_string.decode('unicode_escape')
byte_string = escaped_string.encode('latin1')
nparr = np.fromstring(byte_string, np.uint8)
rgb = nparr.reshape((1080, 1920, -1))
plt.imshow(rgb)

grey = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
plt.imshow(grey, cmap = 'gray')


blur = cv2.GaussianBlur(grey, ksize=(25,25), sigmaX=0)
plt.imshow(blur, cmap = 'gray')


thresh, mask = cv2.threshold(blur, thresh = 150, maxval = 175, type = cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
plt.imshow(mask, cmap = 'gray')


edges = cv2.Canny(blur, threshold1 = 200, threshold2 = 255, apertureSize = 5, L2gradient = True)
plt.imshow(edges, cmap = 'gray')



edges_and_mask = cv2.bitwise_and(edges, mask)
plt.imshow(edges_and_mask)
lines = cv2.HoughLinesWithAccumulator(edges_and_mask, rho = 5, theta = 0.09, threshold = 150) 
lines = np.squeeze(lines)
# sort by max votes
sorted_lines = lines[(-lines[:, 2]).argsort()]
for i in range(sorted_lines.shape[0]):
    rho = sorted_lines[i, 0]
    theta = sorted_lines[i, 1]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv2.line(rgb, pt1, pt2, (0,0,255), 2)
plt.imshow(rgb)

rho_clusters = fclusterdata(sorted_lines[:, 0].reshape(-1, 1), t = 5, criterion = 'distance', method = 'complete')
theta_clusters = fclusterdata(sorted_lines[:, 1].reshape(-1, 1), t = 0.09, criterion = 'distance', method = 'complete')

best_lines = []
checked_clusters = []
for i in range(sorted_lines.shape[0]):
    rho_cluster = rho_clusters[i]
    theta_cluster = theta_clusters[i]
    cluster = (rho_cluster, theta_cluster)
    if (cluster in checked_clusters):
        continue
    best_lines.append([lines[i, 0], lines[i, 1]])
    checked_clusters.append(cluster)
    if len(best_lines) >= 2:
        break

best_lines = np.asarray(best_lines)

for i in range(best_lines.shape[0]):
    rho = best_lines[i, 0]
    theta = best_lines[i, 1]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 2000*(-b)), int(y0 + 2000*(a)))
    pt2 = (int(x0 - 2000*(-b)), int(y0 - 2000*(a)))
    cv2.line(rgb, pt1, pt2, (255,0,0), 2)
plt.imshow(rgb)

In [1]:
import yaml
import os
import numpy as np


# construct node
def opencv_matrix(loader, node):
    mapping = loader.construct_mapping(node, deep=True)
    mat = np.array(mapping["data"])
    mat.resize(mapping["rows"], mapping["cols"])
    return mat
yaml.add_constructor(u"tag:yaml.org,2002:opencv-matrix", opencv_matrix)

# loading
with open('journal_dataset/camera_calibration.yaml') as fin:
    c = fin.read()
    # some operator on raw conent of c may be needed
    c = "%YAML 1.1"+os.linesep+"---" + c[len("%YAML:1.0"):] if c.startswith("%YAML:1.0") else c
    result = yaml.full_load(c)

In [2]:
result

{'K1': array([[1.64355164e+03, 0.00000000e+00, 8.33842175e+02],
        [0.00000000e+00, 1.63281196e+03, 6.57989555e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'K2': array([[1.63577915e+03, 0.00000000e+00, 1.05825488e+03],
        [0.00000000e+00, 1.63112678e+03, 6.94985444e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'D1': array([[-0.40444239,  0.58161898, -0.00497978,  0.00232176, -0.21547479]]),
 'D2': array([[-0.38302513,  0.49543626, -0.00251143,  0.00254013, -0.05051454]]),
 'ImageSize': [1080, 1920],
 'T': [-5.30606823731539, -0.203133361927184, -0.256298572991149],
 'R': array([[ 0.99988655, -0.00130389, -0.01500622],
        [ 0.00111945,  0.99992382, -0.01229268],
        [ 0.01502111,  0.01227449,  0.99981183]])}

In [3]:
# loading
with open('journal_dataset/handeye.yaml') as fin:
    c = fin.read()
    # some operator on raw conent of c may be needed
    c = "%YAML 1.1"+os.linesep+"---" + c[len("%YAML:1.0"):] if c.startswith("%YAML:1.0") else c
    result = yaml.full_load(c)

In [4]:
result

{'PSM1_rvec': [0.9103082418441772, -2.4375524520874023, 1.2004119157791138],
 'PSM1_tvec': [95.79222106933594, -58.37983703613281, -1.4819064140319824],
 'cam_rvec': [0.0, 0.0, 0.0],
 'cam_tvec': [0.0, 0.0, 0.0]}

In [7]:
# loading
with open('journal_dataset/camera_calibration.yaml') as fin:
    c = fin.read()
    # some operator on raw conent of c may be needed
    c = "%YAML 1.1"+os.linesep+"---" + c[len("%YAML:1.0"):] if c.startswith("%YAML:1.0") else c
    result = yaml.full_load(c)

In [8]:
result

{'K1': array([[1.64355164e+03, 0.00000000e+00, 8.33842175e+02],
        [0.00000000e+00, 1.63281196e+03, 6.57989555e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'K2': array([[1.63577915e+03, 0.00000000e+00, 1.05825488e+03],
        [0.00000000e+00, 1.63112678e+03, 6.94985444e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'D1': array([[-0.40444239,  0.58161898, -0.00497978,  0.00232176, -0.21547479]]),
 'D2': array([[-0.38302513,  0.49543626, -0.00251143,  0.00254013, -0.05051454]]),
 'ImageSize': [1080, 1920],
 'T': [-5.30606823731539, -0.203133361927184, -0.256298572991149],
 'R': array([[ 0.99988655, -0.00130389, -0.01500622],
        [ 0.00111945,  0.99992382, -0.01229268],
        [ 0.01502111,  0.01227449,  0.99981183]])}

In [10]:
result['K1'].shape

(3, 3)

In [1]:
import numpy as np
std = np.asarray([1.0e-3, 1.0e-3, 1.0e-3, # pos
                1.0e-2, 1.0e-2, 1.0e-2, # ori
                0.0, 0.0, 0.0])   # joints

In [6]:
from scipy.stats import multivariate_normal

In [ ]:
sample = multivariate_normal(x = np.zeros(std.shape[0]), cov = np.diag(std), )

In [4]:
np.diag(std)

array([[0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.01 , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.01 , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.01 , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

In [5]:
np.random.multivariate_normal(mean = np.zeros(std.shape[0]), cov = np.diag(std), size = 1)

array([[ 0.00959427,  0.01783143, -0.04159998, -0.05011424,  0.03354048,
         0.0069199 ,  0.        ,  0.        ,  0.        ]])